In [4]:
%%bash
dev=/home/peterr/macocu/task5_webgenres/data/final/fasttext1/dev_onlykeep_False_onlyprimary_True_dedup_False.fasttext
test=/home/peterr/macocu/task5_webgenres/data/final/fasttext1/dev_onlykeep_False_onlyprimary_True_dedup_False.fasttext

destination=/home/peterr/macocu/task5_webgenres/data/final/fasttext1/devtest_onlykeep_False_onlyprimary_True_dedup_False.fasttext

cat $dev $test > $destination


In [14]:
import os
import parse
import fasttext
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
datadir = "/home/peterr/macocu/task5_webgenres/data/final/fasttext1"

dev_full = os.path.join(datadir, "dev_onlykeep_False_onlyprimary_True_dedup_False.fasttext")
test_full = os.path.join(datadir, "test_onlykeep_False_onlyprimary_True_dedup_False.fasttext")
train_full = os.path.join(datadir, "train_onlykeep_False_onlyprimary_True_dedup_False.fasttext")
devtest_full = os.path.join(datadir, "devtest_onlykeep_False_onlyprimary_True_dedup_False.fasttext")

train_labels = ['__label__Legal/Regulation',
 '__label__Opinionated_News',
 '__label__News/Reporting',
 '__label__Forum',
 '__label__Correspondence',
 '__label__Invitation',
 '__label__Instruction',
 '__label__Recipe',
 '__label__Opinion/Argumentation',
 '__label__Promotion_of_Services',
 '__label__Promotion',
 '__label__List_of_Summaries/Excerpts',
 '__label__Promotion_of_a_Product',
 '__label__Call',
 '__label__Review',
 '__label__Other',
 '__label__Information/Explanation',
 '__label__Interview',
 '__label__Prose',
 '__label__Research_Article',
 '__label__Announcement']

STR_TO_NUM = {s: i for i, s in enumerate(train_labels)}
NUM_TO_STR = {i: s for i, s in enumerate(train_labels)}


def parse_fasttext_file(path: str, encode=True):
    """Reads fasttext formatted file and returns dataframe."""
    with open(path, "r") as f:
        content = f.readlines()
    pattern = "{label} {text}\n"
    p = parse.compile(pattern)

    labels, texts = list(), list()
    for line in content:
        rez = p.parse(line)
        if rez is not None:
            labels.append(rez["label"])
            texts.append(rez["text"])
        else:
            print("error parsing line ", line)
    if encode:
        labels = [STR_TO_NUM[i] for i in labels]
    return pd.DataFrame(data={"text": texts, "labels": labels})

train_df = parse_fasttext_file(train_full)
eval_df = parse_fasttext_file(dev_full)
test_df = parse_fasttext_file(devtest_full)

In [15]:
test_df.labels.unique().shape

(19,)

In [16]:
train_df.labels.unique().shape

(21,)

In [17]:
from simpletransformers.classification import ClassificationModel
model_args = {
    "num_train_epochs": 10,
    "learning_rate": 1e-5,
    "overwrite_output_dir": True,
    "train_batch_size": 32,
    "no_save": True,
    "no_cache": True,
    "overwrite_output_dir": True,
    "save_steps": -1,
    "max_seq_length": 256
}

model = ClassificationModel(
    "camembert", "EMBEDDIA/sloberta",
    num_labels = 21,
    use_cuda = True,
    args = model_args
)
model.train_model(train_df)

Some weights of the model checkpoint at EMBEDDIA/sloberta were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
